In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, metrics
import gc
import os

Because we have already did data preprocessing and feature engineering, here we only focus on the models 

LGBM Model

In [2]:
### read the data and split the train & test according to date
data = pd.read_csv("../m5-forecasting-data/train_data.csv")
x_train = data[data['date'] <= '2016-03-27']
y_train = x_train['sell_price']
x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
y_val = x_val['sell_price']
test = data[(data['date'] > '2016-04-24')]

In [9]:
from lightgbm import LGBMRegressor

### train the lightgbm
model = LGBMRegressor( n_estimators=50, learning_rate=0.1, subsample=0.75, max_depth=10 )
model.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_val,y_val)], eval_metric='rmse')
valid_preds[X_valid.index] = model.predict(x_val)
eval_preds[X_test.index] = model.predict(X_test)

val_pred = model.predict(x_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_pred = model.predict(test[features])

### predict part
predictions = test[['id', 'date', 'sell_prices']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sell_prices').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('accu_lgbm_submission.csv', index = False)


[100]	training's rmse: 2.3869	valid_1's rmse: 2.13703
[200]	training's rmse: 2.335	valid_1's rmse: 2.13161
[300]	training's rmse: 2.30045	valid_1's rmse: 2.12539
[400]	training's rmse: 2.27377	valid_1's rmse: 2.12436
[500]	training's rmse: 2.25216	valid_1's rmse: 2.12296
[600]	training's rmse: 2.23511	valid_1's rmse: 2.12073
Early stopping, best iteration is:
[647]	training's rmse: 2.22812	valid_1's rmse: 2.11932
Our val rmse score is 2.1193242240937646



LSTM Model

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, metrics
from ipywidgets import widgets, interactive
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta 
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
import datetime as dt
import random 
import os
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time 
from sklearn.metrics import mean_squared_error
%matplotlib inline

from fastprogress import master_bar, progress_bar
import torch 
from torch.autograd import Variable
from torch.utils.data import Dataset
import torch
import torch.nn as nn

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])


In [8]:
train_size = int(len(x_train) * 0.67)
test_size = len(x_train) - train_size

x = x_train
y = x_train['sell_price']
dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [10]:
class myLSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,dropout = 0.25)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        out = self.dropout(out)
    
        return out

In [15]:
#####  Parameters  ######################
num_epochs = 500
learning_rate = 1e-3
input_size = 1
hidden_size = 512
num_layers = 1
num_classes = 1

model = myLSTM(num_classes, input_size, hidden_size, num_layers)

##### Optimzer and scheduler ####################
criterion = torch.nn.MSELoss().to(device)    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience=500,factor =0.5 ,min_lr=1e-7, eps=1e-08)

# Training
for epoch in range(num_epochs): 
    model.train()
    outputs = lstm(trainX)
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    
    #Evaluate on test     
    lstm.eval()
    valid = lstm(testX.to(device))
    vall_loss = criterion(valid, testY.to(device))
    scheduler.step(vall_loss)
    
    if epoch % 50 == 0:
        print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch, loss.cpu().item(),vall_loss.cpu().item()))

Epoch: 0, loss: 0.38196 valid loss:  0.40848 
Epoch: 50, loss: 0.17385 valid loss:  0.11541 
Epoch: 100, loss: 0.16984 valid loss:  0.10561 
Epoch: 150, loss: 0.16380 valid loss:  0.09598 
Epoch: 200, loss: 0.16025 valid loss:  0.10714 
Epoch: 250, loss: 0.15275 valid loss:  0.11765 
Epoch: 300, loss: 0.14970 valid loss:  0.10055 
Epoch: 350, loss: 0.15538 valid loss:  0.09523 
Epoch: 400, loss: 0.15899 valid loss:  0.09488 
Epoch: 450, loss: 0.15080 valid loss:  0.09012 


In [16]:
## submission 
timesteps = 14
X_test = []
X_test.append(test[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30491))
    testInput = np.column_stack((np.array(predicted_stock_price), pd.DataFrame(np.zeros((1969,1)))[1941:1969][0][1941 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j + 1,30491)
    sc = MinMaxScaler(feature_range = (0, 1))
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490)).T
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv(dataPath + "/sample_submission.csv")
idColumn = sample_submission[["id"]]
submission[["id"]] = idColumn  
cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]
submission.columns = colsdeneme
submission.to_csv("accu_lstm_submission.csv", index=False)